In [9]:
import pandas as pd 
import glob

pd.set_option('display.max_columns', None)

In [ ]:
clients = pd.read_csv('processed/clients.csv', encoding='utf-8')
transactions = pd.read_csv('processed/all_transactions.csv', encoding='utf-8')
transfers = pd.read_csv('processed/all_transfers.csv', encoding='utf-8')

# dropping direction column from transfers as it's not useful
transfers = transfers.drop(columns=['direction'])
# transactions = transactions.drop(columns=['date'])

# standardize client types
types = {
    'Зарплатный клиент': ['зарплатный клиент', 'зп'],
    'Премиальный клиент': ['премиальный клиент', 'вип'],
    'Студент': ['студент', 'студент'],
    'Стандартный клиент': ['стандартный клиент', 'обычный']
}

clients['status'] = clients['status'].str.lower()
transactions['status'] = transactions['status'].str.lower()
transfers['status'] = transfers['status'].str.lower()

for df in [clients, transactions, transfers]:
    df['status'] = df['status'].str.strip()
    
for df in [clients, transactions, transfers]:
    for standard, variants in types.items():
        df.loc[df['status'].isin(variants), 'status'] = standard
        
print(f'Clients shape: {clients.shape}')
display(clients.sort_values('client_code').head(5))
print(f'Transfers shape: {transfers.shape}')
display(transfers.sort_values('client_code').head(5))
print(f'Transactions shape: {transactions.shape}')
display(transactions.sort_values('client_code').head(5))

Clients shape: (59, 6)


,client_code,name,status,age,city,avg_monthly_balance_KZT
0,1,Айгерим,Зарплатный клиент,29,Алматы,92643
1,2,Данияр,Премиальный клиент,41,Астана,1577073
2,3,Сабина,Студент,22,Алматы,63116
3,4,Тимур,Зарплатный клиент,36,Караганда,83351
4,5,Камилла,Премиальный клиент,45,Алматы,1336536


Transfers shape: (17700, 9)


,client_code,name,product,status,city,date,type,amount,currency
15583,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-08-27 13:52:57,atm_withdrawal,28318.31,KZT
15582,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-08-26 12:23:26,card_out,21578.45,KZT
15581,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-08-26 12:19:08,card_out,23260.79,KZT
15580,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-08-26 09:29:13,p2p_out,14201.30,KZT
15595,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-08-30 15:26:18,card_out,36485.00,KZT


Transactions shape: (17700, 9)


,client_code,name,product,status,city,date,category,amount,currency
5451,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-06-15 19:00:40,Кафе и рестораны,5725.94,KZT
5450,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-06-15 15:50:24,Кино,4286.91,KZT
5449,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-06-15 12:10:12,Продукты питания,14803.48,KZT
5448,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-06-15 10:30:00,Путешествия,39623.64,KZT
5499,1,Айгерим,Карта для путешествий,Зарплатный клиент,Алматы,2025-06-30 18:20:10,Смотрим дома,6813.12,KZT


In [22]:
clients = clients[['client_code', 'name', 'age', 'status', 'avg_monthly_balance_KZT', 'city',]]
transfers_merged = transfers.merge(clients[['client_code']], on='client_code', how='left')
transactions_merged = transactions.merge(clients[['client_code']], on='client_code', how='left')


transfers_aggregated = (
    transfers_merged.groupby(['client_code', 'date', 'type', 'currency'], dropna=False)['amount']
    .sum()
    .reset_index()
    .sort_values(['currency', 'amount'], ascending=[True, False])
)

transactions_aggregated = (
    transactions_merged.groupby(['client_code', 'date', 'category', 'currency'], dropna=False)['amount']
    .sum()
    .reset_index()
    .sort_values(['currency', 'amount'], ascending=[True, False])
)
print(f'Clients shape: {clients.shape}')
display(clients.sort_values('client_code').head(5))
print(f'Transfers shape: {transfers.shape}')
display(transfers_aggregated.sort_values('client_code').head(5))
print(f'Transactions shape: {transactions.shape}')
display(transactions_aggregated.sort_values('client_code').head(5))

Clients shape: (59, 6)


,client_code,name,age,status,avg_monthly_balance_KZT,city
0,1,Айгерим,29,Зарплатный клиент,92643,Алматы
1,2,Данияр,41,Премиальный клиент,1577073,Астана
2,3,Сабина,22,Студент,63116,Алматы
3,4,Тимур,36,Зарплатный клиент,83351,Караганда
4,5,Камилла,45,Премиальный клиент,1336536,Алматы


Transfers shape: (17700, 9)


,client_code,date,type,currency,amount
137,1,2025-07-12 09:39:46,atm_withdrawal,KZT,20432.40
230,1,2025-08-10 11:50:45,card_out,KZT,47004.39
71,1,2025-06-22 20:57:59,card_out,KZT,47020.25
128,1,2025-07-08 14:24:05,card_out,KZT,26817.00
57,1,2025-06-18 20:40:30,p2p_out,KZT,38552.16


Transactions shape: (17700, 9)


,client_code,date,category,currency,amount
233,1,2025-08-10 09:15:22,Такси,KZT,3920.96
15,1,2025-06-07 08:20:04,Такси,KZT,3914.26
271,1,2025-08-25 09:00:39,Кафе и рестораны,KZT,3896.02
244,1,2025-08-15 12:10:44,Продукты питания,KZT,7109.60
290,1,2025-08-29 11:30:23,Смотрим дома,KZT,5222.45


In [ ]:
# 1) which category and how much money is spent on each category for specific client_code
# 2) which type and how much money is spent on each type for specific client_code
# 3) which kinds of currency are used for specific client_code
# 4) what's the avg_monthly_balance_KZT for specific client_code

In [ ]:
print(clients.client_code.unique())
print()

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60]


In [24]:
transactions['product'].unique()

array(['Депозит Мультивалютный', 'Золотые слитки', 'Кредит наличными',
       'Депозит Накопительный', 'Обмен валют', 'Инвестиции',
       'Депозит Сберегательный', 'Карта для путешествий',
       'Кредитная карта', 'Премиальная карта'], dtype=object)